# MCap Performance Index

In [150]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [262]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from mining.modules.assets.etp import performIndex as indx
from utils.modules.ml.timeseries import rollingstats as stats
from utils.modules.etl.load import sparkNoSQLwls as nosql
from utils.modules.lib.spark import execSession as spark

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    indx = importlib.reload(indx)
    stats= importlib.reload(stats)
    nosql= importlib.reload(nosql)
    spark= importlib.reload(spark)
    
__desc__ = "analyze crypto market capitalization time series data"
clsIndx =indx.Portfolio(desc=__desc__)
clsStat=stats.RollingStats(desc=__desc__)
clsNoSQL=nosql.NoSQLWorkLoads(desc=__desc__)
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional PERFORMINDEX-libraries in ETP-package of ASSETS-module imported successfully!
All packages in utils ml timeseries RollingStats imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional EXECSESSION-libraries in SPARK-package of LIB-module imported successfully!
sparkNoSQLwls Class initialization complete
execSession Class initialization complete
sparkNoSQLwls Class initialization complete
dailyTopN Class initialization complete
performIndex Class initialization complete
execSession Class initialization complete
sparkNoSQLwls Class initialization complete

Class initialization and load complete!


## Read Portfolio

In [152]:
_db_coll=["mpt.top3.2022-01-10","mpt.top3.2022-01-09"]
_coll_dt=date(2022,1,10)
_mpt_port = clsIndx.get_portfolio(
    db_name="tip-daily-mpt",
    db_coll=[],#_db_coll,
    coll_date=_coll_dt,
)
_mpt_port

[{'_id': ObjectId('63d4bb80badb4157b3025cb3'),
  'date': '2022-01-10T00:00:00',
  'asset': 'charg_coin',
  'mcap.weight': 0.4453207926976063,
  'mcap.ror': 0.6574680456259401,
  'mcap.value': 1148729.3496556643},
 {'_id': ObjectId('63d4bb80badb4157b3025cb4'),
  'date': '2022-01-10T00:00:00',
  'asset': 'matryx',
  'mcap.weight': 0.34821271988748376,
  'mcap.ror': 0.2861116369010913,
  'mcap.value': 2246173.234597205},
 {'_id': ObjectId('63d4bb80badb4157b3025cb5'),
  'date': '2022-01-10T00:00:00',
  'asset': 'bitcoinpos',
  'mcap.weight': 0.20646648741490986,
  'mcap.ror': 0.20342319564693787,
  'mcap.value': 14522857.416871611}]

In [257]:
clsIndx.data.show()

+----------+----------+--------------------+--------------------+--------------------+--------------------+
| mcap_date|asset_name|          mcap_value| mcap_value_prev_val|           mcap_diff|             log_ror|
+----------+----------+--------------------+--------------------+--------------------+--------------------+
|2021-12-27|    matryx|90564566.46846270...|                null|               0E-15|                null|
|2021-12-27|charg_coin|5165891.154802010...|                null|               0E-15|                null|
|2021-12-27|bitcoinpos|356969638.9413422...|                null|               0E-15|                null|
|2021-12-28|charg_coin|9159213.308334240...|5165891.154802010...|3993322.153532230...| -0.2487131958281502|
|2021-12-28|bitcoinpos|356969638.9413422...|356969638.9413422...|               0E-15|                 0.0|
|2021-12-28|    matryx|90564566.46846270...|90564566.46846270...|               0E-15|                 0.0|
|2021-12-29|bitcoinpos|35696

In [256]:
__idx_type__='adx'
_kwargs={
    "LOGCOLNAME" :'log_ror',
    "PREVALCOLNAME":'mcap_prev_val',
    "DIFFCOLNAME":'mcap_diff',
}
_rsi = clsIndx.get_index(
    portfolio=_mpt_port,
    date=_coll_dt,
    index_type=__idx_type__,
    **_kwargs,
)
print("Completed index computation for %s = %d" % (__idx_type__.upper(),_rsi))

Wait a moment, retrieving data ...


[Error]function <@data.setter> name '__s_fn_id__' is not defined
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezaware/utils/modules/ml/timeseries/rollingstats.py", line 170, in session
    clsSpark.session={}
NameError: name 'clsSpark' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/nuwan/workspace/rezaware/utils/modules/ml/timeseries/rollingstats.py", line 251, in data
    self._data = self.session.createDataFrame(data)
  File "/home/nuwan/workspace/rezaware/utils/modules/ml/timeseries/rollingstats.py", line 179, in session
    logger.error("%s %s \n",__s_fn_id__, err)
NameError: name '__s_fn_id__' is not defined

[Error]function <simple_moving_stats> name '__s_fn_id__' is not defined
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezaware/utils/modules/ml/timeseries/rollingstats.py", line 221, in data
    raise ValueError("Data is of NoneType; cannot be used in

TypeError: %d format: a number is required, not NoneType

# SMA

In [260]:
_mcap_log_ror=clsIndx.data.na.drop(subset=['log_ror'])

In [264]:
kwargs={
    "DATETIMEATTR":'mcap_date',
    "WINLENGTH":7,
    "WINUNIT":'DAY',
}
_sma_sdf=clsStat.simple_moving_stats(
    column='log_ror',   # column name to apply the rolling computation
    stat_op="avg", # stat operation sum, mean or standard deviation
    data=_mcap_log_ror,   # data set
    **kwargs,    # 
)


In [266]:
_sma_sdf.select(["log_ror","rolling_avg_log_ror"]).show()

23/02/01 16:09:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/02/01 16:09:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 1